## Tensor Inspection

This notebook details the process of identifying and tracking the values of tensors in a given network with an example using Mask RCNN. First, we generate a small dataset consisting of a single image from the coco data. We then look at how to track the tensors within Mask RCNN using that image. Finally, we track the gradients that backpropogate through the network.


In [1]:
import sys
import os
import re
from ast import literal_eval
import json
#os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
os.environ['TENSORPACK_FP16'] = 'true'
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow as tf
import tqdm
import numpy as np
import tensorpack.utils.viz as tpviz
from tensorpack import *
from tensorpack.tfutils.common import get_tf_version_tuple
sys.path.append('/mask-rcnn-tensorflow/MaskRCNN')
from model.generalized_rcnn import ResNetFPNModel
from config import finalize_configs, config as cfg
from eval import DetectionResult, predict_image, multithread_predict_dataflow, EvalCallback
from performance import ThroughputTracker, humanize_float
from data import get_eval_dataflow, get_train_dataflow, get_batch_train_dataflow

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.








/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/numba/errors.py:131: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
MODEL = ResNetFPNModel(True)

In [3]:
cfg.DATA.BASEDIR = '/data/small_sample/'
cfg.BACKBONE.WEIGHTS = '/data/pretrained-models/ImageNet-R50-AlignPadding.npz'
tf.set_random_seed(cfg.TRAIN.SEED)
fix_rng_seed(cfg.TRAIN.SEED)
np.random.seed(cfg.TRAIN.SEED)

In [ ]:
train_dataflow = get_batch_train_dataflow(cfg.TRAIN.BATCH_SIZE_PER_GPU)
finalize_configs(is_training=True)

In [ ]:
session_init = get_model_loader(cfg.BACKBONE.WEIGHTS)

At this point, we have our data and the model is setup. In order to export a gradient from the graph, we need to add a print statement to the tensor we want to track. For example, say we want to export the second to last layer of the backbone network (c4). Add this import to the top of the backbone.py file:

```from performance import print_runtime_tensor, print_runtime_tensor_loose_branch```

The, just after the c4 tensor is created in the network, add this line:

```c4 = print_runtime_tensor(\"tensor_c4_forward\", c4)```

Similarly, say we want to output a list of tensors. Perhaps the full output of the fpn (p23456). We can use something like:

```p23456 = [print_runtime_tensor(\"tensor_p23456_{}_forward\".format(i), j) for i,j in enumerate(p23456)]```

On the other hand, we might want to see a tensor that isn't actually used later in the graph, which means it wouldn't normally execute such that we can output it. This can be done using the function:

`print_runtime_tensor_loose_branch`

For this, you need a downstream trigger tesnor to force the print of the tensor of interest. Say we have a tensor `t5` that isn't used in the graph, but `t1` is. We can print `t5` with:

```t1 = print_runtime_tensor_loose_branch(\"tensor_t5_forward\", t5, trigger_tensor=t1)```

Finally, say we want to print the gradients of the backwards pass. This is a little more complicated. Add this gradient printer class to the generalized_rcnn.py file:

```
class GradientPrinter(tf.train.Optimizer):
    def __init__(self, opt):
        self.opt = opt
    def compute_gradients(self, *args, **kwargs):
        return self.opt.compute_gradients(*args, **kwargs)
    def apply_gradients(self, gradvars, global_step=None, name=None):
        old_grads, v = zip(*gradvars)
        old_grads = [print_runtime_tensor("tensor_{}_backward".format(i.name), j) for i,j in zip(v, old_grads)]
        gradvars = list(zip(old_grads, v))
        return self.opt.apply_gradients(gradvars, global_step, name)
```

Inside the detection model class, modify the optimizer to pass through the gradient printer.

```
opt = tf.train.MomentumOptimizer(lr, 0.9)
opt = GradientPrinter(opt)
```

Once the print function has been added, run the paragraph below with the capture magic function to catch the printed output.

### Detailed example

Let's work through all the steps of tracking a few tensors in the graph, using the fpn example above. First, open the `generalized_rcnn.py` file in the mask rcnn model. 

Add the import for to print a runtime tensor.

<img src="assets/import_printer.png" style="width: 600px;">

At the end of the fpn, add the print_runtime_tensor function. Remember that this function returns the same tensor (which is necessary to put the print command in the graph). Also, the p23456 object output from the fpn is a list, so we can print all tensors in the list with

<img src="assets/print_comprehension.png" style="width: 800px;">

If you want to get more information on this tensor, you can also find it in the graph using tensorboard. Based on the fpn file, these tensors follow the naming convention `posthoc_3x3_p{}` for p2-5 while p6 is named with `max_pool`. To use tensorboard, navigate to the log directory of you model, then select tensorboard with current directory from the new menu in the upper right corner.

<img src="assets/start_tensorboard.png" style="width: 800px;">

This will open tensorboard in a new tab. It might take a few seconds to load, since this is a large model. Once it's loaded you can search `posthoc_3x3` or `max_pool` under the graphs tab to find the fpn. The entire fpn subgraph looks like

<img src="assets/tensorboard.png" style="width: 800px;">

Before we run the model and get these tensors, let's also add in a print function for the backwards pass, so we can get the gradients. This is a little trickier, because it requires including a new optimizer, but you can follow the example from above. In the `generalized_rcnn.py` file, add this to the section just below the GradientClipOptimizer

<img src="assets/add_optimizer.png" style="width: 800px;">

Then include the optimizer just after the initial model optimizer.

<img src="assets/call_optimizer.png" style="width: 800px;">

Now when you run the model, the gradients in p23456 and all backward pass gradients will print to stdout. This is a lot of data that we would rather not have printing as the model runs, so within this notebook, we can supress and capture the output by adding `%%capture cap_out --no-stderr` at the top of the cell running the model. Also, it's a good idea to initially only run a single forward and backward pass by setting `steps_per_epoch` and `max_epoch` to 1.

<img src="assets/model_config.png" style="width: 600px;">


In [6]:
traincfg = TrainConfig(
            model=MODEL,
            data=QueueInput(train_dataflow),
            steps_per_epoch=1,
            max_epoch=1,
            session_init=session_init,
            session_config=None,
            starting_epoch=cfg.TRAIN.STARTING_EPOCH
        )



In [7]:
trainer = SimpleTrainer()

In [ ]:
%%capture cap_out --no-stderr
launch_train_with_config(traincfg, trainer)

As long as you followed the convention of `tensor_[name]_forward` and `tensor_[name]_backward` the two cells below will split the output, and convert the tensors from strings to numeric lists. Note that it's generally a bad idea to print out these lists in the notebook, as they can be extremely large, and cause the notebook to crash. Instead, they can be written to an output file. 

Note that these two paragraphs can take a few minutes to run, as the tensors they parse can be quite large.

In [12]:
forward_tensors = {i.split("_forward ")[0] : \
                   literal_eval(re.sub("\s+", ", ", i.split("_forward ")[1])) \
                   for i in tqdm.tqdm([j for j in \
                                       cap_out.stdout.split("[runtime_tensor] tensor_")[1:] \
                                       if "_forward " in j])}

100%|██████████| 5/5 [02:03<00:00, 24.76s/it]


In [14]:
backward_tensors = {i.split("_backward ")[0] : \
                    literal_eval(re.sub("\s+", ", ", i.split("_backward ")[1])) \
                    for i in tqdm.tqdm([j for j in \
                                        cap_out.stdout.split("[runtime_tensor] tensor_")[1:] \
                                        if "_backward " in j])}

100%|██████████| 168/168 [04:56<00:00,  1.77s/it]


Let's take a look at some statistics about the tensors.

In [20]:
print("Forward Tensor Shape")
for i,j in forward_tensors.items():
    print("{} : {}".format(i, np.array(j).shape))
    
print("\nBackward Tensor Shape")
for i,j in backward_tensors.items():
    print("{} : {}".format(i, np.array(j).shape))

Forward Tensor Shape
p6 : (1, 1, 256, 13, 19)
p5 : (1, 1, 256, 25, 37)
p4 : (1, 1, 256, 50, 74)
p3 : (1, 1, 256, 100, 148)
p2 : (1, 1, 256, 200, 296)

Backward Tensor Shape
rpn/box/b:0 : (1, 12)
rpn/box/W:0 : (1, 1, 1, 256, 12)
rpn/class/b:0 : (1, 3)
rpn/conv0/b:0 : (1, 256)
rpn/class/W:0 : (1, 1, 1, 256, 3)
fastrcnn/outputs/box/b:0 : (1, 324)
fastrcnn/outputs/class/b:0 : (1, 81)
fastrcnn/fc7/b:0 : (1, 1024)
fastrcnn/fc6/b:0 : (1, 1024)
maskrcnn/conv/b:0 : (1, 80)
maskrcnn/deconv/b:0 : (1, 256)
maskrcnn/fcn3/b:0 : (1, 256)
maskrcnn/fcn2/b:0 : (1, 256)
maskrcnn/fcn1/b:0 : (1, 256)
maskrcnn/fcn0/b:0 : (1, 256)
maskrcnn/conv/W:0 : (1, 1, 1, 256, 80)
fastrcnn/outputs/class/W:0 : (1, 1024, 81)
fpn/posthoc_3x3_p3/b:0 : (1, 256)
fpn/posthoc_3x3_p2/b:0 : (1, 256)
fpn/posthoc_3x3_p4/b:0 : (1, 256)
fpn/posthoc_3x3_p5/b:0 : (1, 256)
fpn/lateral_1x1_c2/b:0 : (1, 256)
fpn/lateral_1x1_c3/b:0 : (1, 256)
fpn/lateral_1x1_c4/b:0 : (1, 256)
fpn/lateral_1x1_c5/b:0 : (1, 256)
group3/block2/conv3/bn/beta:0 

In [23]:
print("Forward Tensor Means")
for i,j in forward_tensors.items():
    print("{} : {}".format(i, np.array(j).mean()))
    
print("\nBackward Tensor Means")
for i,j in backward_tensors.items():
    print("{} : {}".format(i, np.array(j).mean()))

Forward Tensor Means
p6 : -0.018399720606300633
p5 : -0.020339489301798633
p4 : -0.021849885086021547
p3 : 0.007539847410840255
p2 : 0.01815363196401787

Backward Tensor Means
rpn/box/b:0 : -0.008242289228333333
rpn/box/W:0 : -0.003379818523977779
rpn/class/b:0 : 0.14231363933333332
rpn/conv0/b:0 : -1.7733052107812532e-05
rpn/class/W:0 : 0.06580069587422252
fastrcnn/outputs/box/b:0 : -2.2287111288487657e-05
fastrcnn/outputs/class/b:0 : 2.8485185125440257e-10
fastrcnn/fc7/b:0 : -2.4970532128134714e-05
fastrcnn/fc6/b:0 : 9.733545261774611e-05
maskrcnn/conv/b:0 : -0.0005258560187499999
maskrcnn/deconv/b:0 : 0.002508225385300781
maskrcnn/fcn3/b:0 : 0.0011247704243133204
maskrcnn/fcn2/b:0 : 0.001305727025769375
maskrcnn/fcn1/b:0 : 0.0010997915820076952
maskrcnn/fcn0/b:0 : 0.0012305637350459766
maskrcnn/conv/W:0 : -0.0006620706915804658
fastrcnn/outputs/class/W:0 : 1.4277089529434338e-09
fpn/posthoc_3x3_p3/b:0 : 4.9572554652070304e-05
fpn/posthoc_3x3_p2/b:0 : 0.0002707897238558594
fpn/postho

group3/block1/conv3/W:0 : -1.0473209368506655e-05
group1/block1/conv2/W:0 : -0.0005154865667325206
group3/block0/conv1/W:0 : 0.00018656335364061494
fpn/lateral_1x1_c5/W:0 : -0.0002007116821010212
fpn/posthoc_3x3_p5/W:0 : 1.7069067191772739e-06
group3/block0/convshortcut/W:0 : 8.965862277156866e-05
group3/block0/conv2/W:0 : 5.533696939667889e-05
group3/block2/conv2/W:0 : 6.098815632628358e-05
group3/block1/conv2/W:0 : -8.88479320450196e-05
fastrcnn/fc6/W:0 : 4.526923293837606e-07


Finally, we can save the tensors to json files.

In [24]:
with open('/logs/forward_p23456.json', 'w') as outfile:
    json.dump(forward_tensors, outfile)
    
with open('/logs/backward.json', 'w') as outfile:
    json.dump(backward_tensors, outfile)